In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from numpy import *

# Imports
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from gensim.models import word2vec

[nltk_data] Downloading package stopwords to /home/aman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
y_train = train.age.values
data = pd.concat((train, test)).reset_index(drop=True)
data.drop(['age'], axis=1, inplace=True)

In [15]:
data['year'] = data['date'].str[-4:]

In [14]:
mylen = np.vectorize(len)
data['length_text'] = mylen(data['text'])

In [16]:
data.head()

,date,gender,post.id,sign,text,topic,user.id,year,length_text
0,"14,May,2004",male,1,Leo,"Info has been found (+/- 100 pages,...",Student,11869,2004,157
1,"13,May,2004",male,2,Leo,These are the team members: Drewe...,Student,11869,2004,181
2,"12,May,2004",male,3,Leo,In het kader van kernfusie op aarde...,Student,11869,2004,25467
3,"12,May,2004",male,4,Leo,testing!!! testing!!!,Student,11869,2004,43
4,"11,June,2004",male,5,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,InvestmentBanking,16332,2004,402


In [30]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
sentiment = pd.DataFrame(columns=('Positive', 'Negative', 'Neutral', 'Compound'))
from nltk.tokenize import sent_tokenize, word_tokenize
counter = 0
for text in data['text']:
    text_compound = 0
    text_neg = 0
    text_pos = 0
    text_neu = 0
    counter = counter + 1
    try:
        sentences = sent_tokenize(text)
        l = len(sentences)
    except:
        print (counter)
        sentences = ['test', 'test2 jh']
        l = 1000000000000
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        text_compound = text_compound + vs['compound']
        text_neg = text_neg + vs['neg']
        text_pos = text_pos + vs['pos']
        text_neu = text_neu + vs['neu']
    if l == 0:
        l = 10000000000000
    text_sentiment = pd.DataFrame([[text_pos/l, text_neg/l, text_neu/l, text_compound/l]], columns=('Positive', 'Negative', 'Neutral', 'Compound'))
    sentiment = sentiment.append(text_sentiment, ignore_index = True)

In [40]:
final_data = pd.concat([data, sentiment], axis=1)

In [41]:
final_data.head()

,date,gender,post.id,sign,text,topic,user.id,year,length_text,Positive,Negative,Neutral,Compound
0,"14,May,2004",male,1,Leo,"Info has been found (+/- 100 pages,...",Student,11869,2004,157,0.000000,0.000000,1.000000,0.000000
1,"13,May,2004",male,2,Leo,These are the team members: Drewe...,Student,11869,2004,181,0.000000,0.000000,1.000000,0.000000
2,"12,May,2004",male,3,Leo,In het kader van kernfusie op aarde...,Student,11869,2004,25467,0.081101,0.070965,0.847934,0.003812
3,"12,May,2004",male,4,Leo,testing!!! testing!!!,Student,11869,2004,43,0.000000,0.000000,0.666667,0.000000
4,"11,June,2004",male,5,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,InvestmentBanking,16332,2004,402,0.430500,0.000000,0.569500,0.393100


In [42]:
one_hot_columns = ['gender', 'sign', 'topic', 'year']
one_hot_columns = pd.get_dummies(final_data[one_hot_columns], dummy_na=True)

In [44]:
final_data = pd.concat([final_data, one_hot_columns], axis=1)
final_data.to_csv('data_with_sentiment_score_and_one_hot.csv', encoding='utf-8')

In [45]:
final_data.columns

Index(['date', 'gender', 'post.id', 'sign', 'text', 'topic', 'user.id', 'year',
       'length_text', 'Positive', 'Negative', 'Neutral', 'Compound',
       'gender_female', 'gender_male', 'gender_nan', 'sign_Aquarius',
       'sign_Aries', 'sign_Cancer', 'sign_Capricorn', 'sign_Gemini',
       'sign_Leo', 'sign_Libra', 'sign_Pisces', 'sign_Sagittarius',
       'sign_Scorpio', 'sign_Taurus', 'sign_Virgo', 'sign_nan',
       'topic_Accounting', 'topic_Advertising', 'topic_Agriculture',
       'topic_Architecture', 'topic_Arts', 'topic_Automotive', 'topic_Banking',
       'topic_Biotech', 'topic_BusinessServices', 'topic_Chemicals',
       'topic_Communications-Media', 'topic_Construction', 'topic_Consulting',
       'topic_Education', 'topic_Engineering', 'topic_Environment',
       'topic_Fashion', 'topic_Government', 'topic_HumanResources',
       'topic_Internet', 'topic_InvestmentBanking', 'topic_Law',
       'topic_LawEnforcement-Security', 'topic_Manufacturing',
       'topic_Marit

In [58]:
train_columns = ['length_text', 'Positive', 'Negative', 'Neutral', 'Compound',
       'gender_female', 'gender_male', 'gender_nan', 'sign_Aquarius',
       'sign_Aries', 'sign_Cancer', 'sign_Capricorn', 'sign_Gemini',
       'sign_Leo', 'sign_Libra', 'sign_Pisces', 'sign_Sagittarius',
       'sign_Scorpio', 'sign_Taurus', 'sign_Virgo', 'sign_nan',
       'topic_Accounting', 'topic_Advertising', 'topic_Agriculture',
       'topic_Architecture', 'topic_Arts', 'topic_Automotive', 'topic_Banking',
       'topic_Biotech', 'topic_BusinessServices', 'topic_Chemicals',
       'topic_Communications-Media', 'topic_Construction', 'topic_Consulting',
       'topic_Education', 'topic_Engineering', 'topic_Environment',
       'topic_Fashion', 'topic_Government', 'topic_HumanResources',
       'topic_Internet', 'topic_InvestmentBanking', 'topic_Law',
       'topic_LawEnforcement-Security', 'topic_Manufacturing',
       'topic_Maritime', 'topic_Marketing', 'topic_Military',
       'topic_Museums-Libraries', 'topic_Non-Profit', 'topic_Publishing',
       'topic_RealEstate', 'topic_Religion', 'topic_Science',
       'topic_Sports-Recreation', 'topic_Student', 'topic_Technology',
       'topic_Telecommunications', 'topic_Tourism', 'topic_Transportation',
       'topic_indUnk', 'topic_nan', 'year_,,', 'year_1999', 'year_2000',
       'year_2001', 'year_2002', 'year_2003', 'year_2004', 'year_2005',
       'year_2006', 'year_nan']

train_data = final_data[train_columns][:len(train['text'])]
test_data = final_data[train_columns][len(train['text']):]

In [46]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [64]:
lasso =  make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
cross_val_score(lasso, train_data, train['age'], cv=4)

array([0.16039372, 0.17801303, 0.12669821, 0.20925074])

In [ ]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=8, learning_rate=0.1)
cross_val_score(xgbm, train_data, train['age'], cv=4)

In [68]:
xgbm.fit(train_data, train['age'])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=360,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [85]:
preds = xgbm.predict(test_data)
solution = pd.DataFrame({"user.id":test["user.id"], "age":preds})

In [71]:
solution = solution.groupby('user.id', as_index=False)['age'].mean()

In [72]:
solution.to_csv("solutions/metadata_sentiment_xgbm.csv", index = False)

In [73]:
lasso.fit(train_data, train['age'])

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [77]:
preds_lasso = lasso.predict(test_data)
solution_lasso = pd.DataFrame({"user.id":test["user.id"], "age":preds})
solution_lasso = solution_lasso.groupby('user.id', as_index=False)['age'].mean()
solution_lasso.to_csv("solutions/metadata_sentiment_lasso.csv", index = False)

In [89]:
ensamble = (preds_lasso + preds)/2
solution_ensamble = pd.DataFrame({"user.id":test["user.id"], "age":ensamble})
solution_ensamble = solution_ensamble.groupby('user.id', as_index=False)['age'].mean()
solution_ensamble.to_csv("solutions/metadata_sentiment_ensamble.csv", index = False)

In [87]:
preds_lasso

array([23.72066564, 24.66381767, 22.13861759, ..., 22.01031735,
       22.06346438, 22.25570712])